IMT 2200 - Introducción a Ciencia de Datos<br>
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>

---

## Proyecto de Investigación – Terremotos en Chile


## Pregunta 5:
## **Modelo predictivo de lugares de riesgo**<br>

In [1]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
from os import chdir, getcwd
if 'Preguntas'in getcwd():
    chdir('../')

In [4]:
df = pd.read_csv('data//query_limpio.csv', index_col=0)
df.columns

Index(['level_0', 'index', 'latitude', 'longitude', 'depth', 'mag', 'magType',
       'nst', 'gap', 'dmin', 'rms', 'id', 'horizontalError', 'depthError',
       'magError', 'magNst', 'date', 'hour', 'zone', 'region'],
      dtype='object')

In [ ]:
x = df[[]]